In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/rice-type-classification/riceClassification.csv


In [2]:
path='/kaggle/input/rice-type-classification/riceClassification.csv'

In [3]:
import torch 
import torch.nn as nn 
from torch.optim import Adam 
from torch.utils.data import Dataset , DataLoader
from torchsummary import summary 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
import matplotlib as plt 
import pandas as pd 
import numpy as np 

In [4]:
df=pd.read_csv(path)
df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [5]:
df.dropna(inplace=True)
df.drop(['id'],axis=1 ,inplace=True)
print(df.shape)

(18185, 11)


In [6]:
original_df=df.copy()

for col in df.columns:
    df[col]=df[col]/df[col].abs().max()
df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [7]:
X =np.array(df.iloc[:,:-1])
Y=np.array(df.iloc[:,-1])

In [8]:
X_train, X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)

In [9]:
X_test,X_val,y_test,y_val=train_test_split(X_test,y_test,test_size=0.5)

In [10]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(12729, 10)
(2728, 10)
(2728, 10)


In [11]:
class dataset(Dataset):
    def __init__(self,X,Y):
        self.X=torch.tensor(X,dtype=torch.float32)
        self.Y=torch.tensor(Y,dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,index):
        return self.X[index],self.Y[index]

training_data=dataset(X_train,y_train)
testing_data=dataset(X_test,y_test)
validating_data=dataset(X_val,y_val)
        

In [12]:
BATCH_SIZE=32
EPOCHS=10
HIDDEN_NEURONS=10
LR=1e-3

In [13]:
train_dataloader=DataLoader(training_data,batch_size=BATCH_SIZE,shuffle=True)
test_dataloader=DataLoader(testing_data,batch_size=BATCH_SIZE,shuffle=True)
val_dataloader=DataLoader(validating_data,batch_size=BATCH_SIZE,shuffle=True)


Model Class

In [14]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel,self).__init__()
        self.input_layer=nn.Linear(X.shape[1],HIDDEN_NEURONS)
        self.linear=nn.Linear(HIDDEN_NEURONS,1)
        self.sigmoid=nn.Sigmoid()
    
    def forward(self,x):
        x=self.input_layer(x)
        x=self.linear(x)
        x=self.sigmoid(x)
        return x

Model Creation 

In [15]:
model=MyModel()
summary(model,(X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
            Linear-2                    [-1, 1]              11
           Sigmoid-3                    [-1, 1]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


Loss And Optimizer
loss we are Binary Cross Entropy and Optimizer is Adam 

In [16]:
criterion=nn.BCELoss()
optimizer=Adam(model.parameters(),lr=LR)

In [17]:
total_loss_train_plot = []
total_loss_validation_plot = []
total_acc_train_plot = []
total_acc_validation_plot = []

for epoch in range(EPOCHS):
    total_acc_train = 0
    total_loss_train = 0
    total_acc_val = 0
    total_loss_val = 0
    ## Training and Validation
    for data in train_dataloader:

        inputs, labels = data

        prediction = model(inputs).squeeze(1)

        batch_loss = criterion(prediction, labels)

        total_loss_train += batch_loss.item()

        acc = ((prediction).round() == labels).sum().item()

        total_acc_train += acc

        batch_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    ## Validation
    with torch.no_grad():
        for data in val_dataloader:
            inputs, labels = data

            prediction = model(inputs).squeeze(1)

            batch_loss = criterion(prediction, labels)

            total_loss_val += batch_loss.item()

            acc = ((prediction).round() == labels).sum().item()

            total_acc_val += acc

    total_loss_train_plot.append(round(total_loss_train/1000, 4))
    total_loss_validation_plot.append(round(total_loss_val/1000, 4))
    total_acc_train_plot.append(round(total_acc_train/(training_data.__len__())*100, 4))
    total_acc_validation_plot.append(round(total_acc_val/(validating_data.__len__())*100, 4))

    print(f'''Epoch no. {epoch + 1} Train Loss: {total_loss_train/1000:.4f} Train Accuracy: {(total_acc_train/(training_data.__len__())*100):.4f} Validation Loss: {total_loss_val/1000:.4f} Validation Accuracy: {(total_acc_val/(validating_data.__len__())*100):.4f}''')
    print("="*50)

Epoch no. 1 Train Loss: 0.2484 Train Accuracy: 74.5934 Validation Loss: 0.0419 Validation Accuracy: 98.1305
Epoch no. 2 Train Loss: 0.1289 Train Accuracy: 97.8867 Validation Loss: 0.0175 Validation Accuracy: 97.4340
Epoch no. 3 Train Loss: 0.0592 Train Accuracy: 98.3502 Validation Loss: 0.0096 Validation Accuracy: 98.3871
Epoch no. 4 Train Loss: 0.0372 Train Accuracy: 98.4445 Validation Loss: 0.0070 Validation Accuracy: 98.5704
Epoch no. 5 Train Loss: 0.0284 Train Accuracy: 98.5545 Validation Loss: 0.0056 Validation Accuracy: 98.4604
Epoch no. 6 Train Loss: 0.0239 Train Accuracy: 98.5859 Validation Loss: 0.0049 Validation Accuracy: 98.4971
Epoch no. 7 Train Loss: 0.0213 Train Accuracy: 98.5545 Validation Loss: 0.0045 Validation Accuracy: 98.6437
Epoch no. 8 Train Loss: 0.0197 Train Accuracy: 98.5859 Validation Loss: 0.0042 Validation Accuracy: 98.6804
Epoch no. 9 Train Loss: 0.0188 Train Accuracy: 98.6330 Validation Loss: 0.0041 Validation Accuracy: 98.6804
Epoch no. 10 Train Loss: 0.0

In [18]:
with torch.no_grad():
    total_loss_test = 0
    total_acc_test = 0
    for data in test_dataloader:
        inputs , labels =data
        prediction = model(inputs).squeeze(1)
        batch_loss_test=criterion((prediction),labels)
        total_loss_test+= batch_loss_test.item()
        acc = ((prediction).round() == labels).sum().item()
        total_acc_test+=acc

print(f"Accuracy Score is: {round((total_acc_test/X_test.shape[0])*100, 2)}%")


Accuracy Score is: 98.79%
